In [73]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, SequentialSampler,DataLoader
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm


In [11]:
data_dir = 'data/'

In [64]:
class SlidingWindowDataset(Dataset):
    """Sliding window dataset"""
    
    def __init__(self,csv_data,timesteps,sliding_window):
        self.data = csv_data
        self.timesteps = timesteps
        self.sliding_window = sliding_window
    
    def __getitem__(self,index):
        print(index)
        x = self.data.iloc[:, index: index + self.timesteps]
        y = self.data.iloc[:, index + self.timesteps : index+self.sliding_window ]
        x = torch.tensor(x.values)
        y = torch.tensor(y.values)
        return x,y
    
    def __len__(self):
        return self.data.shape[1] - self.sliding_window + 1
        
        

In [65]:
# read data (for now, sell_prices & calendar are not used)

data_dir = '../code/data/'

train_sales = pd.read_csv(data_dir + 'sales_train_validation.csv')
#sell_prices = pd.read_csv(data_dir + 'sell_prices.csv')
#calendar = pd.read_csv(data_dir + 'calendar.csv')
submission_file = pd.read_csv(data_dir + 'sample_submission.csv')

In [66]:
# create training data, for now it only contains the sales and no extra features
sales = train_sales.drop(["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1)
sales.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [67]:
start_day = 1

timesteps = 14
prediction_steps = 28
len_window = timesteps + prediction_steps



In [68]:
dataset = SlidingWindowDataset(sales,14,(timesteps+prediction_steps))

In [69]:
dataset_sampler = SequentialSampler(dataset)

In [70]:
dataset_sampler_loader = DataLoader(dataset,batch_size=32,shuffle=False)

In [73]:
## I think it would be easier to select a certain set of features, 
## train it with a CNN and extract feacture vectors for the LSTM
class EventCNN(nn.Module):
    def __init__(self, n_features):
        super(EventCNN, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(input_shape, 256)
        self.feature_linear = nn.Linear(256, n_features)
        self.classification_linear = nn.Linear(n_features, nr_of_events)
        
    def forward(self, x):
        h = self.linear_1(self.flatten(x))
        h = self.feature_linear(h)
        h = self.classification_linear(h)
        return torch.squeeze(h)

In [8]:
class LSTM(nn.Module):
    def __init__(self,prediction_steps):
        super(LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=1,hidden_size=10,num_layers=2,bidirectional=True)
        self.lin = nn.Linear(10,28)
        
    def forward(self,x):
        x = self.lstm1(x)
        x = self.lin(x)
        return x
    

In [75]:
class CombinationNet(nn.Module):
    
    def __init__(self,cnn,lstm):
        super(CombinationNet, self).__init__()
        self.feature_vectors = torch.nn.Sequential(*(list(cnn_net.children())[:-1]))
        self.lstm = lstm
    
    def forward(self,x):
        x = self.feature_vectors(x)
        x = self.lstm(x)
        

In [76]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(in_channels, 64, (2,1),1 ) ## input depends on sliding window...
        self.relu = nn.Relu()
        self.pool = nn.MaxPool2d((2,1),2)
        self.timedistributed = ... ### not sure about this one, this is a keras specific layer, but we could just use a linear layer
        self.lstm = nn.LSTM(64,hidden_size, num_layers=1)
        self.tanh = nn.Tanh
        self.lin1 = nn.Linear(in_features,32)
        self.lin2 = nn.Linear(in_features,output_size) # 
    
    def forward(self,x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        ## TimeDistributed
        x = self.lstm(x)
        x = self.tanh(x)
        x = self.lin1(x)
        x = self.lin2(x)
        return x

In [75]:
lstm = LSTM(28)

In [76]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001,weight_decay=1e-5)


In [79]:
for batch_id, (sliding_window,label) in tqdm(enumerate(dataset_sampler_loader), total=len(dataset_sampler_loader), desc="#train batches", leave=False):
    lstm.train()
    outputs = lstm(sliding_window)
    optimizer.zero_grad()
    loss = criterion(outputs,label)
    
    loss.backward()
    optimizer.step()


    print("Epoch: %d, loss: %1.5f  " %(epoch, loss.cpu().item()))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


RuntimeError: input.size(-1) must be equal to input_size. Expected 1, got 14